In [1]:
%reset -f
from parflow import Run
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb, read_pfb
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import TensorDataset
from utils import init_location, cal_z_locid, cal_velz_loc, \
                  interpolate_vel, data_normalize, To_Tensor
# from parflow.tools.hydrology import calculate_water_table_depth, \
# calculate_overland_flow_grid, calculate_overland_fluxes
# %matplotlib widget

In [2]:
p_start = 35040
t_start = 0 #iteration start
t_end = 40 #iteration end
real_path = 'False'

npcell = 10  #used for train
testsize = 1  #used for test
npcell = npcell + testsize  #all
# how many particles in a grid cell

interval = 1 #always no gap
step_size = 1 #always one hour

In [3]:
# input model parameters
nx = 20
ny = 5
nz = 20

dx = 5
dy = 0.2
dz = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, \
               0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.3, 0.1])
boundings = np.array([0., 100., 0., 1., 0., 9.4])

In [4]:
for ii in range(t_start,t_end):
    
    istep = 1 + ii #current iterate step
    print('istep',istep)

    Vx = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velx.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Vy = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.vely.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Vz = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velz.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Saturation = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.satur.' \
                                            +str(istep+p_start).zfill(5)+'.pfb'))
    Porosity = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.porosity.pfb'))
    # print(Vx.shape, Vy.shape, Vz.shape)

    if real_path == 'False' or (istep == t_start + 1 and real_path == 'True'):
        #setup initial location
        initx,inity,initz = init_location(npcell,nx,ny,nz,dx,dy,dz)
        # initz[1,0,:,:]

        #cal locid
        Px = np.copy(initx)
        Py = np.copy(inity)
        Pz = np.copy(initz)
        
        print('real_path',real_path)
    
    if istep != t_start + 1 and real_path == 'True':
        initx = np.copy(Px)
        inity = np.copy(Py)
        initz = np.copy(Pz)
        
        print('real_path',real_path)

    Plocx = np.floor(Px/dx)
    Plocy = np.floor(Py/dy)
    Plocz = cal_z_locid(npcell,nx,ny,nz,dz,Pz)

    # cal vel loc
    # this can be got by random number directly
    # but for multiple steps, we still need to calculate
    # so we provide the template here
    # we may also calculate it from random number
    # i mean like add the intial fraction with the new_distance/vel
    # but additional other calculation may need

    Clocx = (Px - Plocx*dx)/dx
    Clocy = (Py - Plocy*dy)/dy
    Clocz = cal_velz_loc(npcell,nx,ny,nz,dz,Pz,Plocz)

    Vpx, Vpy, Vpz = interpolate_vel(npcell,nx,ny,nz,Plocx,Plocy,Plocz,Clocx,Clocy,Clocz, \
                                   Vx,Vy,Vz,Saturation,Porosity)

    Px = Px + Vpx
    Py = Py + Vpy
    Pz = Pz + Vpz

    tt,initx,inity,initz,Px,Py,Pz = data_normalize(istep,interval,step_size,t_start,t_end, \
                                                   initx,inity,initz,Px,Py,Pz,boundings)

    start_train_new, end_train_new, time_train_new = To_Tensor(0,npcell-testsize,nx,ny,nz,initx,inity,initz,Px,Py,Pz,tt)
    start_test_new, end_test_new, time_test_new = To_Tensor(npcell-testsize,npcell,nx,ny,nz,initx,inity,initz,Px,Py,Pz,tt)
    
    if istep == t_start + 1:
        start_train = start_train_new
        end_train   = end_train_new
        time_train  = time_train_new
        
        start_test  = start_test_new
        end_test    = end_test_new
        time_test   = time_test_new
        
    else:
        
        start_train = torch.cat((start_train, start_train_new),dim=0)
        end_train   = torch.cat((end_train, end_train_new),dim=0)
        time_train  = torch.cat((time_train, time_train_new),dim=0)
        
        start_test  = torch.cat((start_test, start_test_new),dim=0)
        end_test    = torch.cat((end_test, end_test_new),dim=0)
        time_test   = torch.cat((time_test, time_test_new),dim=0)

istep 1
real_path False
istep 2
real_path False
istep 3
real_path False
istep 4
real_path False
istep 5
real_path False
istep 6
real_path False
istep 7
real_path False
istep 8
real_path False
istep 9
real_path False
istep 10
real_path False
istep 11
real_path False
istep 12
real_path False
istep 13
real_path False
istep 14
real_path False
istep 15
real_path False
istep 16
real_path False
istep 17
real_path False
istep 18
real_path False
istep 19
real_path False
istep 20
real_path False
istep 21
real_path False
istep 22
real_path False
istep 23
real_path False
istep 24
real_path False
istep 25
real_path False
istep 26
real_path False
istep 27
real_path False
istep 28
real_path False
istep 29
real_path False
istep 30
real_path False
istep 31
real_path False
istep 32
real_path False
istep 33
real_path False
istep 34
real_path False
istep 35
real_path False
istep 36
real_path False
istep 37
real_path False
istep 38
real_path False
istep 39
real_path False
istep 40
real_path False


In [8]:
data_train = TensorDataset(start_train,end_train,time_train)
data_test = TensorDataset(start_test,end_test,time_test)

In [6]:
torch.save(data_train,'data_train.pth')
torch.save(data_test,'data_test.pth')

In [9]:
count = 0
for x in data_train:#generator
    print(x)
    count = count + 1
    if count > 10: break

(tensor([-0.9045, -0.6214, -0.9085]), tensor([-0.9046, -0.6214, -0.9084]), tensor([-0.9500]))
(tensor([-0.8883, -0.9401, -0.9543]), tensor([-0.8884, -0.9401, -0.9543]), tensor([-0.9500]))
(tensor([-0.7216, -0.8252, -0.9748]), tensor([-0.7217, -0.8252, -0.9748]), tensor([-0.9500]))
(tensor([-0.6613, -0.9579, -0.9624]), tensor([-0.6614, -0.9579, -0.9624]), tensor([-0.9500]))
(tensor([-0.5734, -0.9281, -0.9235]), tensor([-0.5735, -0.9281, -0.9235]), tensor([-0.9500]))
(tensor([-0.4884, -0.7740, -0.9482]), tensor([-0.4885, -0.7740, -0.9482]), tensor([-0.9500]))
(tensor([-0.3908, -0.9325, -0.9647]), tensor([-0.3909, -0.9325, -0.9647]), tensor([-0.9500]))
(tensor([-0.2318, -0.7606, -0.9363]), tensor([-0.2319, -0.7606, -0.9363]), tensor([-0.9500]))
(tensor([-0.1292, -0.9018, -0.9174]), tensor([-0.1293, -0.9018, -0.9174]), tensor([-0.9500]))
(tensor([-0.0350, -0.6687, -0.9213]), tensor([-0.0351, -0.6687, -0.9213]), tensor([-0.9500]))
(tensor([ 0.0490, -0.6279, -0.9313]), tensor([ 0.0489, -0.62